### Hugging Face Accelerate Demo

**Note**: Before running this demo, please make sure that you have `wandb.ai` free account. 

Let us install `accelerate`.

In [1]:
!pip install accelerate

**Import** the required modules.

In [2]:
import torch
import torchvision
import wandb
import datetime
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from ui import progress_bar

# This is a demo of the PyTorch Accelerate API.
from accelerate import Accelerator

**`wandb`** initialization. See [`wandb_demo`](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/versions/2022/tools/python/wandb_demo.ipynb) notebook for more details.


In [3]:
wandb.login()
config = {
  "learning_rate": 0.1,
  "epochs": 100,
  "batch_size": 128,
  "dataset": "cifar10"
}
run = wandb.init(project="accelerate-project", entity="upeee", config=config)

wandb: Currently logged in as: rowel (use `wandb login --relogin` to force relogin)
2022-03-12 20:54:48.217804: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-12 20:54:48.217847: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Build the model

Use a ResNet18 from `torchvision`. See [`wandb_demo`](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/versions/2022/tools/python/wandb_demo.ipynb) notebook for more details.

In [4]:
# Shows the code to be replaced with the Accelerate API.
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
accelerator = Accelerator()

model = torchvision.models.resnet18(pretrained=False, progress=True)

model.fc = torch.nn.Linear(model.fc.in_features, 10) 

# Replace the model with the Accelerate API.
#model.to(device)

# watch model gradients during training
wandb.watch(model)

[]

### Loss function, Optimizer, Scheduler and DataLoader

See [`wandb_demo`](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/versions/2022/tools/python/wandb_demo.ipynb) notebook for more details.


In [5]:
loss = torch.nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=wandb.config.learning_rate)
scheduler = CosineAnnealingLR(optimizer, T_max=wandb.config.epochs)

x_train = datasets.CIFAR10(root='./data', train=True, 
                           download=True, 
                           transform=transforms.ToTensor())
x_test = datasets.CIFAR10(root='./data',
                          train=False, 
                          download=True, 
                          transform=transforms.ToTensor())
train_loader = DataLoader(x_train,
                          batch_size=wandb.config.batch_size,
                          shuffle=True,
                          num_workers=2)
test_loader = DataLoader(x_test, 
                         batch_size=wandb.config.batch_size, 
                         shuffle=False, 
                         num_workers=2)

# Accelerate API
model = accelerator.prepare(model)
optimizer = accelerator.prepare(optimizer)
scheduler = accelerator.prepare(scheduler)
train_loader = accelerator.prepare(train_loader)


Files already downloaded and verified
Files already downloaded and verified


### Visulaizing sample data from test split


See [`wandb_demo`](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/versions/2022/tools/python/wandb_demo.ipynb) notebook for more details.


Note the last line that uses Accelerate API to wrap the model, optimizer, data loaders and scheduler.

In [6]:

label_human = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

table_test = wandb.Table(columns=['Image', "Ground Truth", "Initial Pred Label",])

image, label = iter(test_loader).next()
test_loader = accelerator.prepare(test_loader)
image = image.to(accelerator.device)
model.eval()
with torch.no_grad():
  pred = torch.argmax(model(image), dim=1).cpu().numpy()

for i in range(8):
  table_test.add_data(wandb.Image(image[i]),
                      label_human[label[i]], 
                      label_human[pred[i]])
  print(label_human[label[i]], "vs. ",  label_human[pred[i]])

cat vs.  bird
ship vs.  bird
ship vs.  bird
airplane vs.  bird
frog vs.  bird
frog vs.  bird
automobile vs.  bird
frog vs.  bird


### The train loop

Using Accelerate, we do not need to transfer the model to the `device`.

See [`wandb_demo`](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/versions/2022/tools/python/wandb_demo.ipynb) notebook for more details.

In [7]:
def train(epoch):
  model.train()
  train_loss = 0
  correct = 0
  train_samples = 0

  # sample a batch. compute loss and backpropagate
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    # Replaced by the Accelerate API.
    #target = target.to(device)
    #output = model(data.to(device))
    
    output = model(data)
    loss_value = loss(output, target)

    # Replaced by the Accelerate API.
    #loss_value.backward()
    accelerator.backward(loss_value)

    optimizer.step()
    scheduler.step(epoch)
    train_loss += loss_value.item()
    train_samples += len(data)
    pred = output.argmax(dim=1, keepdim=True)
    correct += pred.eq(target.view_as(pred)).sum().item()
    if batch_idx % 10 == 0:
      accuracy = 100. * correct / len(train_loader.dataset)
      progress_bar(batch_idx,
                   len(train_loader),
                   'Train Epoch: {}, Loss: {:.6f}, Acc: {:.2f}%'.format(epoch+1, 
                   train_loss/train_samples, accuracy))
  
  train_loss /= len(train_loader.dataset)
  accuracy = 100. * correct / len(train_loader.dataset)

  return accuracy, train_loss

### The validation loop

After every epoch, we will run the validation loop for the model. Again, no need to transfer the data to the `device`.

See [`wandb_demo`](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/versions/2022/tools/python/wandb_demo.ipynb) notebook for more details.

In [8]:
def test():
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:

      # Replaced by the Accelerate API.
      #output = model(data.to(device))   
      #target = target.to(device)

      output = model(data)
      test_loss += loss(output, target).item()
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  accuracy = 100. * correct / len(test_loader.dataset)

  print('\nTest Loss: {:.4f}, Acc: {:.2f}%\n'.format(test_loss, accuracy))

  return accuracy, test_loss

### `wandb` plots

Finally, we will use `wandb` to visualize the training progress. 
See [`wandb_demo`](https://github.com/roatienza/Deep-Learning-Experiments/blob/master/versions/2022/tools/python/wandb_demo.ipynb) notebook for more details.

In [9]:
run.display(height=1000)

start_time = datetime.datetime.now()
best_acc = 0
for epoch in range(wandb.config["epochs"]):
    train_acc, train_loss = train(epoch)
    test_acc, test_loss = test()
    if test_acc > best_acc:
        wandb.run.summary["Best accuracy"] = test_acc
        best_acc = test_acc
        accelerator.save(model, "resnet18_best_acc.pth")
    wandb.log({
        "Train accuracy": train_acc,
        "Test accuracy": test_acc,
        "Train loss": train_loss,
        "Test loss": test_loss,
        "Learning rate": optimizer.param_groups[0]['lr']
    })

elapsed_time = datetime.datetime.now() - start_time
print("Elapsed time: %s" % elapsed_time)
wandb.run.summary["Elapsed train time"] = str(elapsed_time)

model.eval()
with torch.no_grad():
  pred = torch.argmax(model(image), dim=1).cpu().numpy()

final_pred = []
for i in range(8):
    final_pred.append(label_human[pred[i]])
    print(label_human[label[i]], "vs. ",  final_pred[i])

table_test.add_column(name="Final Pred Label", data=final_pred)

wandb.log({"Test data": table_test})

wandb.finish()

/home/rowel/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


/home/rowel/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


 [=============================>]  Step: 163ms | Tot: 6s733ms | Train Epoch: 1, Loss: 0.013032, Acc: 44.19 391/391 

Test Loss: 0.0101, Acc: 53.99%

 [=============================>]  Step: 161ms | Tot: 6s574ms | Train Epoch: 2, Loss: 0.008935, Acc: 59.64 391/391 

Test Loss: 0.0130, Acc: 46.23%

 [=============================>]  Step: 167ms | Tot: 6s599ms | Train Epoch: 3, Loss: 0.007254, Acc: 67.17 391/391 

Test Loss: 0.0120, Acc: 52.00%

 [=============================>]  Step: 167ms | Tot: 6s612ms | Train Epoch: 4, Loss: 0.006131, Acc: 72.55 391/391 

Test Loss: 0.0113, Acc: 55.62%

 [=============================>]  Step: 161ms | Tot: 6s553ms | Train Epoch: 5, Loss: 0.005271, Acc: 76.14 391/391 

Test Loss: 0.0121, Acc: 52.32%

 [=============================>]  Step: 162ms | Tot: 6s721ms | Train Epoch: 6, Loss: 0.004501, Acc: 79.66 391/391  

Test Loss: 0.0114, Acc: 58.94%

 [=============================>]  Step: 161ms | Tot: 6s583ms | Train Epoch: 7, Loss: 0.003756, Acc: 83.1

 [=============================>]  Step: 165ms | Tot: 6s676ms | Train Epoch: 55, Loss: 0.000000, Acc: 100.00 391/391 

Test Loss: 0.0167, Acc: 74.94%

 [=============================>]  Step: 162ms | Tot: 6s586ms | Train Epoch: 56, Loss: 0.000000, Acc: 100.00 391/391 

Test Loss: 0.0167, Acc: 74.90%

 [=============================>]  Step: 161ms | Tot: 6s770ms | Train Epoch: 57, Loss: 0.000000, Acc: 100.00 391/391 

Test Loss: 0.0166, Acc: 74.95%

 [=============================>]  Step: 161ms | Tot: 6s607ms | Train Epoch: 58, Loss: 0.000000, Acc: 100.00 391/391 

Test Loss: 0.0166, Acc: 75.05%

 [=============================>]  Step: 162ms | Tot: 6s617ms | Train Epoch: 59, Loss: 0.000000, Acc: 100.00 391/391 

Test Loss: 0.0168, Acc: 74.82%

 [=============================>]  Step: 162ms | Tot: 6s644ms | Train Epoch: 60, Loss: 0.000000, Acc: 100.00 391/391 

Test Loss: 0.0168, Acc: 75.02%

 [=============================>]  Step: 162ms | Tot: 6s573ms | Train Epoch: 61, Loss: 0.00000

Learning rate,████████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
Test accuracy,▂▁▃▅▅▅▇▅▇▅▆▇████████████████████████████
Test loss,▂▃▃▁▃▄▃▆▅█▇▅▅▅▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
Train accuracy,▁▄▅▆▇▇██████████████████████████████████
Train loss,█▅▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Best accuracy,75.19
Elapsed train time,0:13:03.392685
Learning rate,2e-05
Test accuracy,75.16
Test loss,0.01693
Train accuracy,100.0


### Load the best performing model

In the following code, we load the best performing model. The model is saved in `./resnet18_best_acc.pth`. The average accuracy of the model is the same as the one in the previous section.

In [10]:
model = torch.load("resnet18_best_acc.pth")
# Using Accelerator API
model = accelerator.prepare(model)
accuracy, _ = test()
print("Best accuracy: %.2f" % accuracy)


Test Loss: 0.0167, Acc: 75.19%

Best accuracy: 75.19
